In [ ]:
################# Dependencies

In [9]:
import os
import openai
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator
from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index import download_loader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding

from llama_index import  ServiceContext
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index import ServiceContext, LLMPredictor
import datetime
from llama_index import StorageContext, load_index_from_storage
from llama_index import GPTVectorStoreIndex
import json
import os
import glob
JsonDataReader = download_loader("JsonDataReader")


In [10]:
####################Variables, objects creation

In [12]:
os.environ['OPENAI_API_KEY'] = "ad OPANAI token...."

In [ ]:
# Setting parameters for GPT3.5 model 
openai.api_key = os.environ['OPENAI_API_KEY']
gpt_35_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3))
SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = SimpleInputPrompt(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST]"
)

In [ ]:
########### Loading index and querying

In [13]:
llm=OpenAI(model='gpt-3.5-turbo', temperature=0.3)

In [14]:
import os
#current_path = os.getcwd()
path = os.path.dirname(os.getcwd())

#path = os.path.dirname(current_path)
service_context = ServiceContext.from_defaults(llm=llm)
storage_context = StorageContext.from_defaults(persist_dir=path+"/data/indexes/GPT3.5_index")
index = load_index_from_storage(storage_context, service_context=service_context)
query_engine = index.as_query_engine()

In [17]:
# Create query

system_prompt = "Act as a Eurostat's statistical assistant. Use the context data provided as descriptions of eurostat tables. Answer questions in a friendly manner, based on the given source documents as well as general statistical eurostat knowledge. Limit the output to the data related only to eurostat. Here are some rules you always follow: (1) Generate human readable output, avoid creating output with gibberish text. Answer directly with no additional introductory text (2) Generate only the requested output, don't include any other language before or after the requested output. (3) Generate professional language typically used in business documents in North America. (4) Never generate offensive or foul language. Please answer the folllowing question:"

question = "which conutries had the highest growth on GDP in last 10 years and in which tables of eurostat data I can find these information? Provide also the names of the tables"

full_request= system_prompt + question
print (f"REQUEST_WITH_SYSTEM-PROMPT:\n{full_request}")
print (f"\nUSER_PROMPT:\n{question}")

REQUEST_WITH_SYSTEM-PROMPT:
Act as a Eurostat's statistical assistant. Use the context data provided as descriptions of eurostat tables. Answer questions in a friendly manner, based on the given source documents as well as general statistical eurostat knowledge. Limit the output to the data related only to eurostat. Here are some rules you always follow: (1) Generate human readable output, avoid creating output with gibberish text. Answer directly with no additional introductory text (2) Generate only the requested output, don't include any other language before or after the requested output. (3) Generate professional language typically used in business documents in North America. (4) Never generate offensive or foul language. Please answer the folllowing question:which conutries had the highest growth on GDP in last 10 years and in which tables of eurostat data I can find these information? Provide also the names of the tables

USER_PROMPT:
which conutries had the highest growth on GD

In [16]:
import datetime
print("start time:", datetime.datetime.now())

response_basic = query_engine.query(question)
print (f"RESPONSE_WITHOUGHT_SYSTEM-PROMPT:\n{response_basic}")
print ("end time:", datetime.datetime.now())

response_full = query_engine.query(full_request)
print (f"\nRESPONSE_WITH_SYSTEM-PROMPT:\n{response_full}")
print ("end time:", datetime.datetime.now())

start time: 2024-01-07 16:42:45.155602
RESPONSE_WITHOUGHT_SYSTEM-PROMPT:
The context information does not provide any specific data or tables related to GDP growth in countries. Therefore, I cannot provide the names of the tables where this information can be found.
end time: 2024-01-07 16:42:48.218548

RESPONSE_WITH_SYSTEM-PROMPT:
The countries with the highest growth in GDP in the last 10 years can be found in the eurostat tables. The specific tables that contain this information are not provided in the given context.
end time: 2024-01-07 16:42:50.836482
